## DATA Preprocess

### Coordinate transform

In [71]:
import numpy as np
import cvxpy as cp

T_z_90 = np.array([[0,-1,0,0],[1,0,0,0],[0,0,1,0],[ 0,0,0,1]])
T_z_min90 = T_z_90.T
R_z_90 = T_z_90[:3, :3]


def coordinate_transform(P0, P1, Pose0, Pose1):
    def Transformation_matrix(Pose0, Pose1):
        # T1 = T0 @ T
        T_matrix = np.linalg.inv(Pose0) @ Pose1 
        # x-axis oriented switched to y-axis oriented
        T_matrix = T_z_90 @ T_matrix @ T_z_min90
        # get transforamtion matrix
        T_matrix = np.linalg.inv(T_matrix)
        return T_matrix
    P0 = (R_z_90 @ P0)
    P1 = (R_z_90 @ P1)
    T_matrix = Transformation_matrix(Pose0, Pose1)
    return P0, P1, T_matrix


### Testing example

In [72]:
theta_Rho = np.array([-0.1764749437570572, 4.030915260314941])
theta_Rho_prime = np.array([-0.21474722027778625, 4.010833740234375])
Pose0 = np.array([[0.980066718449428, 0.19376437086397225, 0.04387021736278988, -1.9997351270147092], [-0.19861530783120357, 0.9607327037625009, 0.19376437086397225, -0.023008590926364158], [-0.004602921125290746, -0.19861530783120357, 0.980066718449428, -0.49986756350735456], [0.0, 0.0, 0.0, 1.0]])
Pose1 =  np.array([[0.9800789348024856, 0.18172971188434028, 0.08012236501040015, -1.9752346722370628], [-0.19362299470144262, 0.9640977376497053, 0.18172971188434028, -0.21569522130654364], [-0.044220102660105357, -0.19362299470144262, 0.9800789348024856, -0.48761733611853136], [0.0, 0.0, 0.0, 1.0]])
P0 = np.array([3.9131364822387695, 0.6978299021720886, 0.6698310375213623])
P1 = np.array([3.8327176570892334, 0.8359556198120117, 0.8356069326400757])

# theta_Rho = np.array([0.65595031, 4.9071393 ])
# theta_Rho_prime = np.array([0.69700497, 4.78944969])
# Pose0 = np.array([[0.9947831471329273, 0.10201220603589002, 0.0, -0.13595595336837749], [-0.10201220603589002, 0.9947831471329273, -0.0, -1.9524370759126866], [-0.0, 0.0, 1.0, -0.3570108264917502], [0.0, 0.0, 0.0, 1.0]])
# Pose1 =  np.array([[0.8676190607260436, -0.49722948973774467, 0.0, 0.7461405519015483], [0.49722948973774467, 0.8676190607260436, 0.0, -0.5554558458716767], [0.0, 0.0, 1.0, -0.37813731018453933], [0.0, 0.0, 0.0, 1.0]])
# P0 = np.array([1.92564058303833, 2.1601450443267822, 0.3570108413696289])
# P1 = np.array([1.3640613555908203, -0.1415318101644516, 0.3781373202800751])



In [73]:
# np.linalg.inv( Pose0 )  @ np.hstack([P0, 1])
P_w = np.array([2, 0, 0, 1])
# Pose0 @ np.hstack([P0, 0])
print(np.dot(np.linalg.inv(Pose0), P_w))
print(np.linalg.inv(Pose0) @ P_w)
print(P0)
print(np.dot(Pose0, np.hstack([P0, 1]).T))

[3.91313657 0.69782992 0.66983106 1.        ]
[3.91313657 0.69782992 0.66983106 1.        ]
[3.91313648 0.6978299  0.66983104]
[ 1.99999991e+00  3.18870264e-10 -1.60621881e-08  1.00000000e+00]


In [74]:
import numpy as np
import cvxpy as cp
from scipy.linalg import eig

T_z_90 = np.array([[0,-1,0,0],[1,0,0,0],[0,0,1,0],[ 0,0,0,1]])
T_z_min90 = T_z_90.T
R_z_90 = T_z_90[:3, :3]

def coordinate_transform_T(Pose0, Pose1):
    # T1 = T0 @ T
    T_matrix = np.linalg.inv(Pose0) @ Pose1 
    # x-axis oriented switched to y-axis oriented
    T_matrix = T_z_90 @ T_matrix @ T_z_min90
    # get transforamtion matrix
    T_matrix = np.linalg.inv(T_matrix)
    return T_matrix

def coordinate_transform_Pose(Pose):
    return (T_z_90 @ Pose @ T_z_min90)

def coordinate_transform_pt(P):
    return (R_z_90 @ P)

def coordinate_transform(P0, P1, Pose0, Pose1):
    P0 = coordinate_transform_pt(P0)
    P1 = coordinate_transform_pt(P1)
    T_matrix = coordinate_transform_T(Pose0, Pose1)
    return P0, P1, T_matrix

P0_new = coordinate_transform_pt(P0)
P1_new = coordinate_transform_pt(P1)
Pose0_new = coordinate_transform_Pose(Pose0)


In [75]:
print(Pose0_new@ np.hstack( [P0_new, 1]))
T_new = coordinate_transform_T(Pose0, Pose1)
P0, P1, T_matrix = coordinate_transform(P0, P1, Pose0, Pose1)

[-3.18870264e-10  1.99999991e+00 -1.60621881e-08  1.00000000e+00]


## Calculate Determinant


In [76]:
def compute_D(T_matrix, theta, theta_prime):
    
    R = T_matrix[:3, :3]
    t = T_matrix[:3, 3]
    
    """
    Compute the determinant D(A0; R, t) for given parameters.
    """
    def skew_symmetric_matrix(t):
        """
        Create a skew-symmetric matrix for a vector t.
        """
        return np.array([
            [0, -t[2], t[1]],
            [t[2], 0, -t[0]],
            [-t[1], t[0], 0]
        ])
    ux = np.array([1, 0, 0])
    uy = np.array([0, 1, 0])
    
    r1 = R[0, :]
    r2 = R[1, :]
        
    t_cross = skew_symmetric_matrix(t)
    
    determinant = - (r1 - np.tan(theta) * r2).T @ t_cross @ (ux - np.tan(theta_prime) * uy)
    
    return determinant

determinant = compute_D(T_matrix, theta=theta_Rho[0], theta_prime=theta_Rho_prime[0])
print("determiant: ", determinant)
##########################################################################


determiant:  0.0008104671265972171


## ANRS

In [77]:

def ANRS(T_matrix, theta_Rho, theta_Rho_prime):

    # 将线性方程组写成矩阵形式 A @ P = B

    R_matrix = T_matrix[:3, :3]
    r1 = R_matrix[0, :]
    r2 = R_matrix[1, :]
    t = T_matrix[:3, 3]


    theta = theta_Rho[0]
    theta_prime = theta_Rho_prime[0]
    R = theta_Rho[1]  # example value for R
    R_prime = theta_Rho_prime[1] # example value for R'
    
    a1 = np.array([-1, np.tan(theta), 0])
    b1 = 0 
    a2 = np.tan(theta_prime) * r2 - r1
    b2 = t[0] - np.tan(theta_prime) * t[1]
    a3 = t.T @ R_matrix
    b3 = (R_prime**2 - R**2 - np.linalg.norm(t)**2) / 2

    A = np.vstack([a1, a2, a3])
    b = np.array([b1, b2, b3])
    ## Testing 
    wanted_result_P = P0
    # print("A @ P = b:\n", (A @ wanted_result_P),"\n should equals to \n", b, "\n" ) # P0 is what we want to get
    
    
    # ANRS
    P_o = np.linalg.inv(A) @ b
    norm_P_o = np.linalg.norm(P_o)
    P_hat_o = P_o / norm_P_o

    A_tilde_prime = np.vstack([A, P_hat_o.T])
    b_tilde_prime = np.append(b, R)

    # 計算 \(\tilde{P}_o'\)
    P_tilde_prime = np.linalg.inv(A_tilde_prime.T @ A_tilde_prime) @ A_tilde_prime.T @ b_tilde_prime      
    
    return P_tilde_prime

ANRS_res = ANRS(T_matrix, theta_Rho, theta_Rho_prime)
print("Calculated P:", ANRS_res)
print("The Ground truth value: ", P0)


Calculated P: [-0.69782896  3.91313266  0.66985476]
The Ground truth value:  [-0.6978299   3.91313648  0.66983104]


## GTRS

In [79]:
import numpy as np
from scipy.linalg import eig



def GTRS(T_matrix, theta_Rho, theta_Rho_prime):

    # Extract components from T_matrix
    R_matrix = T_matrix[:3, :3]
    t = T_matrix[:3, 3]

    # Extract theta, theta_prime, R, R_prime from inputs
    theta = theta_Rho[0]
    theta_prime = theta_Rho_prime[0]
    R = theta_Rho[1]
    R_prime = theta_Rho_prime[1]

    # Define linear constraints in matrix form A @ P = B
    r1 = R_matrix[0, :]
    r2 = R_matrix[1, :]

    a1 = np.array([-1, np.tan(theta), 0, 0])
    b1 = 0 
    a2 = np.tan(theta_prime) * r2 - r1
    a2 = np.hstack([a2, 0])
    b2 = t[0] - np.tan(theta_prime) * t[1]
    a3 = t.T @ R_matrix
    a3 = np.hstack([a3, 0])
    b3 = (R_prime**2 - R**2 - np.linalg.norm(t)**2) / 2
    a4 = np.array([0, 0, 0, 1])
    b4 = R**2

    # 将线性方程组写成矩阵形式 A @ P = B
    A = np.vstack([a1, a2, a3, a4])
    b = np.array([b1, b2, b3, b4])

    ATA_be = A.T @ A
    ATb_be = A.T @ b
    D = np.diag([1,1,1,0])
    g = np.zeros(4)
    g[3] = -0.5

    # eig_lambda = eig(D, ATA_be, right=False)
    eig_lambda = np.real(eig(D, ATA_be, right=False))

    lambda_min = -1 / np.max(eig_lambda)

    # 找二分法的初始左端点
    lambda_l = lambda_min + 2
    y_l = np.linalg.solve(ATA_be + lambda_l * D, ATb_be - lambda_l * g)
    while (y_l.T @ D @ y_l + 2 * g.T @ y_l) < 0:
        lambda_l = (lambda_min + lambda_l) / 2
        y_l = np.linalg.solve(ATA_be + lambda_l * D, ATb_be - lambda_l * g)

    # 找二分法的初始右端点
    lambda_u = lambda_min + 2
    y_u = np.linalg.solve(ATA_be + lambda_u * D, ATb_be - lambda_u * g)
    while (y_u.T @ D @ y_u + 2 * g.T @ y_u) > 0:
        lambda_u = lambda_u + (lambda_u - lambda_min) * 2
        y_u = np.linalg.solve(ATA_be + lambda_u * D, ATb_be - lambda_u * g)

    # 二分法找最优lambda
    while (lambda_u - lambda_l) > 1e-12:
        lambda_temp = (lambda_u + lambda_l) / 2
        y_temp = np.linalg.solve(ATA_be + lambda_temp * D, ATb_be - lambda_temp * g)
        if (y_temp.T @ D @ y_temp + 2 * g.T @ y_temp) > 0:
            lambda_l = lambda_temp
        else:
            lambda_u = lambda_temp

    y = np.linalg.solve(ATA_be + lambda_u * D, ATb_be - lambda_u * g)
    pos = y[:3]
    
    # distance_constraint = y[3] - np.linalg.norm(pos)**2
    # print('Distance constraint value:', distance_constraint)
    
    return pos


# 调用CTOA_GTRS函数
GTRS_res = GTRS(T_matrix, theta_Rho, theta_Rho_prime)

# 显示结果
print('Calculated result:', GTRS_res)
print('Wanted result:', P0)

Calculated result: [-0.69782896  3.91313266  0.66985475]
Wanted result: [-0.6978299   3.91313648  0.66983104]


# Non-linear Optimization

In [67]:
# 定义梯度下降法进行优化
def gradient_descent(P_init, theta_Rho, theta_Rho_prime, T_matrix, learning_rate=0.01, max_iter=1000, tol=1e-5):
    
    def project_to_2d(P):
        X, Y, Z = P
        theta = np.arctan2(X, Y)
        d = np.sqrt(X**2 + Y**2 + Z**2)
        x_s = d * np.sin(theta)
        y_s = d * np.cos(theta)
        return np.array([x_s, y_s])

    # 定义误差函数
    def error_function(P, ps, ps_prime, T_matrix):
        R = T_matrix[:3, :3]
        t = T_matrix[:3, 3]
        # 投影点
        ps_hat = project_to_2d(P)
        P_prime = R @ P + t
        ps_prime_hat = project_to_2d(P_prime)
        
        # 计算误差
        error_ps = ps - ps_hat
        error_ps_prime = ps_prime - ps_prime_hat
        
        # 计算加权误差
        error = error_ps.T @ np.diag(error_ps) @ error_ps + error_ps_prime.T @ np.diag(error_ps_prime) @ error_ps_prime
        return error
    
    theta = theta_Rho[0]
    R = theta_Rho[1]
    theta_prime = theta_Rho_prime[0]
    R_prime = theta_Rho_prime[1]
    ps = np.array([R * np.sin(theta), R * np.cos(theta)])
    ps_prime = np.array([R_prime * np.sin(theta_prime), R_prime * np.cos(theta_prime)])
    
    P = P_init
    for i in range(max_iter):
        # 计算当前误差
        error = error_function(P, ps, ps_prime, T_matrix)
        
        # 计算梯度
        grad = np.zeros_like(P)
        for j in range(len(P)):
            P_temp = P.copy()
            P_temp[j] += tol
            grad[j] = (error_function(P_temp, ps, ps_prime, T_matrix) - error) / tol
        
        # 更新P
        P_new = P + learning_rate * grad
        
        # 检查收敛
        if np.linalg.norm(P_new - P) < tol:
            break
        
        P = P_new
        print(f"Iteration {i+1}, Error: {error}")
    
    return P

In [68]:
P_init = GTRS(T_matrix, theta_Rho, theta_Rho_prime)
P_optimized = gradient_descent(P_init, theta_Rho, theta_Rho_prime, T_matrix)

Iteration 1, Error: 0.0001491524442966133
Iteration 2, Error: 0.0001503007137581909
Iteration 3, Error: 0.00015146065447178524
Iteration 4, Error: 0.00015263241539699126
Iteration 5, Error: 0.00015381614777855895
Iteration 6, Error: 0.0001550120051875327
Iteration 7, Error: 0.00015622014356283408
Iteration 8, Error: 0.00015744072125417445
Iteration 9, Error: 0.00015867389906537832
Iteration 10, Error: 0.00015991984029923006
Iteration 11, Error: 0.00016117871080251293
Iteration 12, Error: 0.00016245067901267182
Iteration 13, Error: 0.00016373591600489545
Iteration 14, Error: 0.00016503459554037493
Iteration 15, Error: 0.0001663468941160118
Iteration 16, Error: 0.0001676729910144143
Iteration 17, Error: 0.00016901306835534306
Iteration 18, Error: 0.00017036731114848815
Iteration 19, Error: 0.00017173590734684312
Iteration 20, Error: 0.00017311904790136013
Iteration 21, Error: 0.0001745169268171508
Iteration 22, Error: 0.0001759297412106552
Iteration 23, Error: 0.00017735769136770173
Iter

In [69]:
print(np.sum(P_init-P0)**2)
print(np.sum(P_optimized-P0)**2)

0.006767541115254317
1.1921617967126936e+28


# Performance Comparation

In [80]:
print(np.sum(ANRS_res-P0)**2)
print(np.sum(GTRS_res-P0)**2)

4.345320807235472e-10
4.3364148520555005e-10


In [8]:
def read_data2(file_path):
    data_sets = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():  # Skip empty lines
                # Parse the line to extract data
                parts = line.split('; ')
                Pose = eval(parts[0].split('Pose: ')[1])
                theta_Rho = eval(parts[1].split('theta_Rho: ')[1])[0]
                P = eval(parts[2].split('s_p: ')[1])[0]
                data_sets.append([Pose, theta_Rho, P])
    return data_sets

file_path = 'record.txt'
data_sets = read_data2(file_path)
times = 0
for i in range(len(data_sets)):
    for j in range(i + 1, len(data_sets)):
        [Pose0, theta_Rho0, P0] = data_sets[i]
        [Pose1, theta_Rho1, P1] = data_sets[j]

        P0, P1, T_matrix = coordinate_transform(P0, P1, Pose0, Pose1)        
        determinant = compute_D(T_matrix, theta_Rho0[0], theta_Rho1[0])
        ANRS_res = ANRS(T_matrix, theta_Rho0, theta_Rho1)
        GTRS_res = GTRS(T_matrix, theta_Rho0, theta_Rho1)
        
        print("Distance: ", np.sqrt( np.sum((P0-P1)**2)))
        print("determinant: ", j-i, determinant)
        print("T_matrix:\n", T_matrix)
        print(np.sum(ANRS_res-P0)**2)
        print(np.sum(GTRS_res-P0)**2)
        print()
        times += 1
        if times > 20:
            break


Distance:  0.23027711287573954
determinant:  1 0.0008104671265972163
T_matrix:
 [[ 0.99990946  0.0124861  -0.00501686 -0.18368821]
 [-0.01266722  0.99920278 -0.03785961 -0.06077924]
 [ 0.00454014  0.03791973  0.99927047  0.02104766]
 [ 0.          0.          0.          1.        ]]
4.3453208083000534e-10
4.3384048744123437e-10

Distance:  0.4007820610673674
determinant:  2 0.0021253403217927997
T_matrix:
 [[ 0.99934816  0.03132265 -0.01794866 -0.33426559]
 [-0.03254454  0.99684447 -0.07240137 -0.14602971]
 [ 0.01562422  0.07293831  0.99721406  0.00459399]
 [ 0.          0.          0.          1.        ]]
7.998121902143994e-12
8.061846437484637e-12

Distance:  0.503310069382022
determinant:  3 0.006286183658378747
T_matrix:
 [[ 0.99775532  0.05509545 -0.03806332 -0.42904985]
 [-0.05873437  0.99302488 -0.10223437 -0.24537478]
 [ 0.03216517  0.10424051  0.99403185 -0.05455015]
 [ 0.          0.          0.          1.        ]]
4.1541986223608696e-14
5.79639329969484e-14

Distance:  0